# 导入工具包

In [1]:
import random
import numpy as np
import pandas as pd
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
# from keras.layers import Lambda, Dense
from keras.layers import *
import tensorflow as tf

Using TensorFlow backend.


# 设置参数

In [2]:
set_gelu('tanh')  # 切换gelu版本

In [3]:
num_classes = 2
maxlen = 128
batch_size = 32
config_path = '../model/albert_small_zh_google/albert_config_small_google.json'
checkpoint_path = '../model/albert_small_zh_google/albert_model.ckpt'
dict_path = '../model/albert_small_zh_google/vocab.txt'

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 加载BERT模型

In [4]:
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='albert',
    return_keras_model=False,
)

# 定义TextCNN网络

## keras辅助函数

In [5]:
expand_dims = Lambda(lambda X:tf.expand_dims(X,axis=-1))
max_pool = Lambda(lambda X:tf.squeeze(tf.reduce_max(X,axis=1),axis=1))
concat = Lambda(lambda X: tf.concat(X, axis=-1))

## 获取bert的char embeeding

In [7]:
cnn_input = expand_dims(bert.layers['Embedding-Token'].output)

## 定义cnn网络

In [8]:
filters = 2
sizes = [3,5,7,9]
output = []
for size_i in sizes:
    X = Conv2D(filters=2,
                   kernel_size=(size_i, 128),
                   activation='relu',
                   )(cnn_input)
    # X = tf.squeeze(tf.reduce_max(X,axis=1),axis=1)
    X = max_pool(X)
    output.append(X)
# cnn_output = tf.concat(output, axis=-1)
cnn_output = concat(output)

## 分类全联接

In [9]:
output = Dense(
    units=num_classes,
    activation='softmax'
)(cnn_output)

## 定义模型输入输出

In [10]:
model = keras.models.Model(bert.model.input[0], output)

## 查看模型结构

In [11]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 128)    2704384     Input-Token[0][0]                
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 128, 1) 0           Embedding-Token[0][0]            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, 1, 2)   770         lambda_1[0][0]                   
____________________________________________________________________________________________

## 编译模型

In [12]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(1e-5),  
    metrics=['accuracy'],
)

# 生成数据

## 加载数据

In [13]:
def load_data(valid_rate=0.3):
    train_file = "../data/train.csv"
    test_file = "../data/test.csv"
    
    df_train_data = pd.read_csv("../data/train.csv").\
    drop_duplicates(['level_1', 'level_2', 'level_3', 'level_4', 'content', 'label'])
    df_test_data = pd.read_csv("../data/test.csv")
    
    train_data, valid_data, test_data = [], [], []
    
    for row_i, data in df_train_data.iterrows():
        id, level_1, level_2, level_3, level_4, content, label = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), label
        if random.random() > valid_rate:
            train_data.append( (id, text, int(label)) )
        else:
            valid_data.append( (id, text, int(label)) )
            
    for row_i, data in df_test_data.iterrows():
        id, level_1, level_2, level_3, level_4, content = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), 0
        test_data.append( (id, text, int(label)) )
    return train_data, valid_data, test_data

In [14]:
train_data, valid_data, test_data = load_data(valid_rate=0.3)

## 迭代器生成

In [15]:
class data_generator(DataGenerator):
    def __iter__(self, random=False):
        batch_token_ids, batch_labels = [], []
        for is_end, (id, text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids], batch_labels
                batch_token_ids, batch_labels = [], []

In [16]:
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)

# 训练、验证和预测

In [17]:
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total

In [18]:
class Evaluator(keras.callbacks.Callback):
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
        test_acc = evaluate(valid_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )

In [19]:
def data_pred(test_data):
    id_ids, y_pred_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        token_ids = sequence_padding([token_ids])
        y_pred = int(model.predict([token_ids]).argmax(axis=1)[0])
        id_ids.append(id)
        y_pred_ids.append(y_pred)
    return id_ids, y_pred_ids

## 训练和验证模型

In [20]:
evaluator = Evaluator()

In [22]:
model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=10,
        callbacks=[evaluator]
    )

Epoch 1/10
234/234 [==============================] - 3s 13ms/step - loss: 0.3866 - accuracy: 0.9233
val_acc: 0.93929, best_val_acc: 0.93929, test_acc: 0.93929

Epoch 2/10
234/234 [==============================] - 3s 11ms/step - loss: 0.3233 - accuracy: 0.9233
val_acc: 0.93929, best_val_acc: 0.93929, test_acc: 0.93929

Epoch 3/10
234/234 [==============================] - 3s 12ms/step - loss: 0.2938 - accuracy: 0.9233
val_acc: 0.93929, best_val_acc: 0.93929, test_acc: 0.93929

Epoch 4/10
234/234 [==============================] - 3s 12ms/step - loss: 0.2795 - accuracy: 0.9233
val_acc: 0.93929, best_val_acc: 0.93929, test_acc: 0.93929

Epoch 5/10
234/234 [==============================] - 3s 11ms/step - loss: 0.2727 - accuracy: 0.9233
val_acc: 0.93929, best_val_acc: 0.93929, test_acc: 0.93929

Epoch 6/10
234/234 [==============================] - 3s 12ms/step - loss: 0.2689 - accuracy: 0.9233
val_acc: 0.93929, best_val_acc: 0.93929, test_acc: 0.93929

Epoch 7/10
234/234 [==============

# 查看模型训练和验证结果

## 加载最好的模型

In [23]:
model.load_weights('best_model.weights')

## 验证集结果

In [24]:
print(u'final test acc: %05f\n' % (evaluate(valid_generator)))

final test acc: 0.939288



## 训练集结果

In [25]:
print(u'final test acc: %05f\n' % (evaluate(train_generator)))

final test acc: 0.923324



# 模型预测保存结果

In [91]:
id_ids, y_pred_ids = data_pred(test_data)
df_save = pd.DataFrame()
df_save['id'] = id_ids
df_save['label'] = y_pred_ids

df_save.to_csv('result.csv')